# Antes de empezar:
- Lee el archivo README.md
- Comenta tanto como puedas y utiliza los recursos (archivo README.md)
- ¡Feliz aprendizaje!

In [1]:
# import numpy and pandas
import pandas as pd
import numpy as np
from scipy.stats import trim_mean, mode, skew, gaussian_kde, pearsonr, spearmanr, beta
from statsmodels.stats.weightstats import ztest as ztest

from scipy.stats import ttest_ind, norm, t
from scipy.stats import f_oneway
from scipy.stats import sem

# Reto 1 - Explorando los Datos

En este reto, examinaremos todos los salarios de los empleados de la Ciudad de Chicago. Comenzaremos cargando el conjunto de datos y examinando su contenido.

In [3]:
# Run this code:
salaries = pd.read_csv('../data/Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examina el conjunto de datos `salaries` utilizando la función `head` que se muestra a continuación.

In [4]:
# Your code here
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


In [12]:
salaries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33183 entries, 0 to 33182
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               33183 non-null  object 
 1   Job Titles         33183 non-null  object 
 2   Department         33183 non-null  object 
 3   Full or Part-Time  33183 non-null  object 
 4   Salary or Hourly   33183 non-null  object 
 5   Typical Hours      8022 non-null   float64
 6   Annual Salary      25161 non-null  float64
 7   Hourly Rate        8022 non-null   float64
dtypes: float64(3), object(5)
memory usage: 2.0+ MB


# Reto 2 - Pruebas de Hipótesis

En esta sección del laboratorio, vamos a probar si el salario por hora de todos los trabajadores por hora es significativamente diferente de $30/hora. Importa la función correcta para la prueba de una muestra desde scipy y realiza la prueba de hipótesis para un intervalo de confianza de dos lados del 95%.

In [23]:
# El DataFrame contiene información de asalariados con salario anual o bien por horas
salaries[(salaries['Salary or Hourly']=='Hourly') & (salaries['Hourly Rate'].isnull())]

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate


In [42]:
# El DataFrame tiene 33183 filas pero sólo 8022 tienen valor non-null en la columna 'Hourly Rate' (coinciden con las filas salaries['Salary or Hourly']=='Hourly')
salaries_hourly = salaries[salaries['Hourly Rate'].notnull()]['Hourly Rate']
salaries_hourly

4        19.86
6        46.10
7        35.60
10        2.65
18       17.68
         ...  
33164    46.10
33168    17.68
33169    35.60
33174    46.35
33175    48.85
Name: Hourly Rate, Length: 8022, dtype: float64

In [62]:
# Prueba de Hipótesis de 1 muestra
from scipy.stats import ttest_1samp
res = ttest_1samp(salaries_hourly, 30)
print(res)
print(f"Estadístico t: {res.statistic:.4f}, Valor p: {res.pvalue:.4e}")

TtestResult(statistic=20.6198057854942, pvalue=4.3230240486229894e-92, df=8021)
Estadístico t: 20.6198, Valor p: 4.3230e-92


In [63]:
# Prueba de Hipótesis Bilateral
# - H0: μ = k
# - H1: μ ≠ k

# Una prueba de hipótesis unilateral solo mira si el estadístico de prueba tiene un valor positivo o negativo muy grande. 
# Una prueba bilateral mira ambos. En el caso bilateral, rechazaremos la hipótesis nula en el caso de que el estadístico de prueba sea un valor positivo o negativo muy grande.

# Típicamente, cuando realizamos una prueba de hipótesis, no podemos estar 100% seguros del resultado. 
# Sin embargo, podemos determinar el porcentaje de certeza en nuestro resultado. 
# Para un 95% de confianza, miramos un 5% de significancia en nuestra prueba.

# Tener un 95% de confianza significa que partimos de la hipótesis nula como el status quo.

alpha = 0.05
resultado = "Rechazamos la hipótesis nula" if res.pvalue < alpha else "No rechazamos la hipótesis nula"
print(f"{resultado}: el salario por hora {'es' if res.pvalue < alpha else 'no es'} significativamente diferente de $30.")

Rechazamos la hipótesis nula: el salario por hora es significativamente diferente de $30.


In [45]:
salaries_hourly.mean()

32.78855771628024

In [46]:
salaries_hourly.describe()

count    8022.000000
mean       32.788558
std        12.112573
min         2.650000
25%        21.200000
50%        35.600000
75%        40.200000
max       109.000000
Name: Hourly Rate, dtype: float64

# Reto 3 - Construyendo Intervalos de Confianza

Aunque probar nuestra hipótesis es una excelente manera de obtener evidencia empírica para aceptar o rechazar la hipótesis, otra forma de recopilar evidencia es creando un intervalo de confianza. Un intervalo de confianza nos brinda información sobre la media verdadera de la población. Por lo tanto, para un intervalo de confianza del 95%, estamos seguros en un 95% de que la media de la población se encuentra dentro del intervalo de confianza.

Para leer más sobre los intervalos de confianza, haz clic [aquí](https://es.wikipedia.org/wiki/Intervalo_de_confianza).

En la celda a continuación, construiremos un intervalo de confianza del 95% para el salario por hora medio de todos los trabajadores por hora.

El intervalo de confianza se calcula en SciPy utilizando la función `t.interval`. Puedes leer más sobre esta función [aquí](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

Para calcular el intervalo de confianza del salario por hora, usa 0.95 para el nivel de confianza, el número de filas - 1 para los grados de libertad, la media de la muestra para el parámetro de ubicación y el error estándar para la escala. El error estándar se puede calcular utilizando [esta](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) función en SciPy.

In [64]:
nivel_de_confianza = 0.95 # alpha
grados_de_libertad = salaries_hourly.count()-1 # df (degrees of freedom) (n-1)
sample_mean = salaries_hourly.mean() # loc (mean)
standard_error = sem(salaries_hourly) # scale (Standard Error of the Mean) (std/sqrt(n))

In [65]:
from scipy.stats import t
res = t.interval(nivel_de_confianza, grados_de_libertad, loc=sample_mean, scale=standard_error)
print('Intervalo de confianza del 95% para el salario por hora medio:', res)

Intervalo de confianza del 95% para el salario por hora medio: (32.52345834488425, 33.05365708767623)


# Reto 4 - Pruebas de Hipótesis de Proporciones

Otro tipo de prueba de una muestra es una prueba de hipótesis de proporciones. En esta prueba, examinamos si la proporción de un grupo en nuestra muestra es significativamente diferente de una fracción.

Puedes leer más sobre las pruebas de proporción de una muestra [aquí](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

En la celda a continuación, utiliza la función `proportions_ztest` de `statsmodels` para realizar una prueba de hipótesis que determinará si el número de trabajadores por hora en la Ciudad de Chicago es significativamente diferente del 25% en el nivel de confianza del 95%.

In [66]:
from statsmodels.stats.proportion import proportions_ztest

count = salaries_hourly.count() # número de trabajadores por hora
nobs = salaries.shape[0]        # total de trabajadores
value = 0.25                    # proporción de trabajadores por hora / total que queremos comprobar
stat, p_value = proportions_ztest(count, nobs, value)
print(f"Estadístico z: {stat:.4f}, Valor p: {p_value:.4e}")

alpha = 0.05
if p_value < alpha:
    print("Rechazamos la hipótesis nula: la proporción de trabajadores por hora es significativamente diferente del 25%.")
else:
    print("No rechazamos la hipótesis nula: no hay evidencia suficiente para decir que la proporción de trabajadores por hora es diferente del 25%.")

Estadístico z: -3.5100, Valor p: 4.4811e-04
Rechazamos la hipótesis nula: la proporción de trabajadores por hora es significativamente diferente del 25%.
